In [ ]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}
settings.MUTATION_PRO = 0.01



class A_3(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_3(GenModel):
    def get_agent_config(self):
        return A_3
    
    def init_env(self):
        #no influence
        return env_settings.inc_env[self.env_val]
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M3'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_SD_LOW = -1 * 2 * settings.INIT_SD
        settings.INIT_SD_HIGH = 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (init_ratio, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = "./res/" self.model_name + "_" + self.specific_kargs_str + '_' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            (TYPE_MAP[0], TYPE_MAP[0] + ' beta', TYPE_MAP[0] + 'gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + ' beta', TYPE_MAP[1] + 'gene'), 'var')+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_popu_gene_mean(self, gen_type):
        return np.mean([np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type])

    def get_popu_gene_var(self, gen_type):
        return np.var([np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type])

    def collect_data(self):
        return (self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(0), self.get_popu_gene_var(0),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(1)), self.get_popu_gene_var(0)
        
        
    def step(self):
        data = self.collect_data()
        print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        super().step()
        
        
